# AILS Publication-Quality Visualizations

This notebook generates all figures for the AILS paper:
- Corridor visualization
- Performance comparison charts
- Scalability analysis plots
- Statistical result figures

**Author:** Amr Elshahed  
**Institution:** Universiti Sains Malaysia

In [ ]:
import sys
sys.path.insert(0, '.')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from datetime import datetime
import os

from AILS_complete import (
    AILSPathfinder, AILSConfig, AILSCorridorBuilder,
    GridGenerator, MovingAIMapLoader
)

# Publication-quality settings
plt.rcParams.update({
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'font.size': 10,
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 9,
    'figure.titlesize': 14,
    'axes.grid': True,
    'grid.alpha': 0.3
})

# Create output directory
FIGURES_DIR = '../manuscript/figures'
os.makedirs(FIGURES_DIR, exist_ok=True)

print("AILS Visualization Notebook")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Output directory: {FIGURES_DIR}")

## 1. Corridor Visualization

In [ ]:
def visualize_corridor(grid, start, goal, corridor, path=None, title="AILS Corridor"):
    """Visualize the AILS corridor on the grid."""
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Create visualization matrix
    vis = np.zeros_like(grid, dtype=float)
    
    # Mark obstacles
    vis[grid == 1] = 1.0
    
    # Mark corridor
    for (r, c) in corridor:
        if 0 <= r < grid.shape[0] and 0 <= c < grid.shape[1]:
            if grid[r, c] == 0:
                vis[r, c] = 0.3
    
    # Mark path
    if path:
        for (r, c) in path:
            vis[r, c] = 0.6
    
    # Mark start and goal
    vis[start] = 0.0
    vis[goal] = 0.0
    
    # Custom colormap
    colors = ['white', 'lightblue', 'blue', 'darkgray']
    cmap = LinearSegmentedColormap.from_list('ails', colors, N=256)
    
    ax.imshow(vis, cmap=cmap, interpolation='nearest')
    
    # Mark start and goal with markers
    ax.plot(start[1], start[0], 'go', markersize=12, label='Start')
    ax.plot(goal[1], goal[0], 'r*', markersize=15, label='Goal')
    
    # Legend
    legend_elements = [
        mpatches.Patch(color='white', label='Traversable'),
        mpatches.Patch(color='darkgray', label='Obstacle'),
        mpatches.Patch(color='lightblue', label='Corridor'),
        mpatches.Patch(color='blue', label='Path'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='g', markersize=10, label='Start'),
        plt.Line2D([0], [0], marker='*', color='w', markerfacecolor='r', markersize=12, label='Goal')
    ]
    ax.legend(handles=legend_elements, loc='upper right', framealpha=0.9)
    
    ax.set_title(title)
    ax.set_xlabel('Column')
    ax.set_ylabel('Row')
    
    return fig, ax

# Generate example
np.random.seed(42)
grid = GridGenerator.generate_clustered(100, density=0.25, num_clusters=10, seed=42)

# Find valid start and goal
traversable = np.argwhere(grid == 0)
idx = np.random.choice(len(traversable), 2, replace=False)
start = tuple(traversable[idx[0]])
goal = tuple(traversable[idx[1]])

# Build corridor
corridor_builder = AILSCorridorBuilder(grid)
corridor = corridor_builder.build_adaptive_corridor(start, goal)

# Find path
pathfinder = AILSPathfinder(grid)
result = pathfinder.find_path_ails(start, goal)

# Visualize
fig, ax = visualize_corridor(grid, start, goal, corridor, result.path,
                             title="AILS Adaptive Corridor Visualization")
fig.savefig(f'{FIGURES_DIR}/corridor_visualization.png', bbox_inches='tight')
plt.show()

print(f"Corridor size: {len(corridor)} cells")
print(f"Path length: {len(result.path)} steps")
print(f"Path cost: {result.cost:.2f}")

## 2. Corridor Comparison (Base vs Adaptive)

In [ ]:
# Compare base and adaptive corridors
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Generate grid with clusters
grid = GridGenerator.generate_clustered(80, density=0.3, num_clusters=8, seed=123)

# Find points
traversable = np.argwhere(grid == 0)
np.random.seed(456)
idx = np.random.choice(len(traversable), 2, replace=False)
start = tuple(traversable[idx[0]])
goal = tuple(traversable[idx[1]])

# Build corridors
corridor_builder = AILSCorridorBuilder(grid)
base_corridor = corridor_builder.build_base_corridor(start, goal)
adaptive_corridor = corridor_builder.build_adaptive_corridor(start, goal)

def plot_corridor_simple(ax, grid, start, goal, corridor, title):
    vis = np.zeros_like(grid, dtype=float)
    vis[grid == 1] = 0.7
    for (r, c) in corridor:
        if 0 <= r < grid.shape[0] and 0 <= c < grid.shape[1] and grid[r, c] == 0:
            vis[r, c] = 0.3
    
    ax.imshow(vis, cmap='Blues_r', interpolation='nearest', vmin=0, vmax=1)
    ax.plot(start[1], start[0], 'go', markersize=10)
    ax.plot(goal[1], goal[0], 'r*', markersize=12)
    ax.set_title(f"{title}\n({len(corridor)} cells)")
    ax.axis('off')

# Plot original grid
axes[0].imshow(grid, cmap='gray_r', interpolation='nearest')
axes[0].plot(start[1], start[0], 'go', markersize=10, label='Start')
axes[0].plot(goal[1], goal[0], 'r*', markersize=12, label='Goal')
axes[0].set_title('Original Grid')
axes[0].axis('off')
axes[0].legend(loc='upper right')

# Plot base corridor
plot_corridor_simple(axes[1], grid, start, goal, base_corridor, 'Base Corridor')

# Plot adaptive corridor
plot_corridor_simple(axes[2], grid, start, goal, adaptive_corridor, 'Adaptive Corridor')

plt.suptitle('AILS Corridor Comparison', fontsize=14, y=1.02)
plt.tight_layout()
fig.savefig(f'{FIGURES_DIR}/corridor_comparison.png', bbox_inches='tight')
plt.show()

## 3. Performance Scaling Plots

In [ ]:
# Generate scaling data
from AILS_complete import run_benchmark, compute_statistics

grid_sizes = [50, 100, 150, 200]
scaling_results = []

for size in grid_sizes:
    print(f"Testing {size}x{size} grid...")
    grid = GridGenerator.generate_random(size, density=0.25, seed=42)
    
    try:
        results = run_benchmark(grid, num_pairs=30, seed=42)
        stats = compute_statistics(results)
        
        for method, s in stats.items():
            scaling_results.append({
                'grid_size': size,
                'method': method,
                'time_mean': s['time_mean'],
                'time_std': s['time_std'],
                'nodes_mean': s['nodes_mean'],
                'nodes_std': s['nodes_std']
            })
    except Exception as e:
        print(f"  Error: {e}")

df_scaling = pd.DataFrame(scaling_results)
print("\nScaling data generated.")

In [ ]:
# Plot scaling results
if len(df_scaling) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    colors = {'standard_astar': 'gray', 'ails_base': 'blue', 'ails_adaptive': 'green'}
    labels = {'standard_astar': 'Standard A*', 'ails_base': 'AILS-Base', 'ails_adaptive': 'AILS-Adaptive'}
    
    # Time plot
    ax = axes[0]
    for method in df_scaling['method'].unique():
        df_m = df_scaling[df_scaling['method'] == method].sort_values('grid_size')
        ax.errorbar(df_m['grid_size'], df_m['time_mean'], yerr=df_m['time_std'],
                   marker='o', capsize=4, label=labels.get(method, method),
                   color=colors.get(method, 'black'))
    ax.set_xlabel('Grid Size (N)')
    ax.set_ylabel('Execution Time (ms)')
    ax.set_title('Search Time vs Grid Size')
    ax.legend()
    ax.set_yscale('log')
    
    # Nodes plot
    ax = axes[1]
    for method in df_scaling['method'].unique():
        df_m = df_scaling[df_scaling['method'] == method].sort_values('grid_size')
        ax.errorbar(df_m['grid_size'], df_m['nodes_mean'], yerr=df_m['nodes_std'],
                   marker='o', capsize=4, label=labels.get(method, method),
                   color=colors.get(method, 'black'))
    ax.set_xlabel('Grid Size (N)')
    ax.set_ylabel('Nodes Visited')
    ax.set_title('Nodes Visited vs Grid Size')
    ax.legend()
    ax.set_yscale('log')
    
    plt.tight_layout()
    fig.savefig(f'{FIGURES_DIR}/scaling_performance.png', bbox_inches='tight')
    plt.show()

## 4. Node Reduction Bar Chart

In [ ]:
# Calculate node reduction percentages
if len(df_scaling) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    reductions = []
    for size in df_scaling['grid_size'].unique():
        df_size = df_scaling[df_scaling['grid_size'] == size]
        std_nodes = df_size[df_size['method'] == 'standard_astar']['nodes_mean'].values
        
        if len(std_nodes) > 0:
            std_nodes = std_nodes[0]
            
            for method in ['ails_base', 'ails_adaptive']:
                ails_nodes = df_size[df_size['method'] == method]['nodes_mean'].values
                if len(ails_nodes) > 0:
                    reduction = (1 - ails_nodes[0] / std_nodes) * 100
                    reductions.append({
                        'grid_size': size,
                        'method': method,
                        'reduction': reduction
                    })
    
    df_red = pd.DataFrame(reductions)
    
    if len(df_red) > 0:
        x = np.arange(len(df_red['grid_size'].unique()))
        width = 0.35
        
        base_red = df_red[df_red['method'] == 'ails_base']['reduction'].values
        adaptive_red = df_red[df_red['method'] == 'ails_adaptive']['reduction'].values
        
        ax.bar(x - width/2, base_red, width, label='AILS-Base', color='steelblue')
        ax.bar(x + width/2, adaptive_red, width, label='AILS-Adaptive', color='forestgreen')
        
        ax.set_xlabel('Grid Size')
        ax.set_ylabel('Node Reduction (%)')
        ax.set_title('Search Space Reduction vs Standard A*')
        ax.set_xticks(x)
        ax.set_xticklabels(df_red['grid_size'].unique())
        ax.legend()
        ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        
        # Add value labels
        for i, (b, a) in enumerate(zip(base_red, adaptive_red)):
            ax.text(i - width/2, b + 1, f'{b:.1f}%', ha='center', fontsize=9)
            ax.text(i + width/2, a + 1, f'{a:.1f}%', ha='center', fontsize=9)
        
        plt.tight_layout()
        fig.savefig(f'{FIGURES_DIR}/node_reduction.png', bbox_inches='tight')
        plt.show()

## 5. Heatmap: Performance by Pattern and Size

In [ ]:
# Generate pattern comparison data
patterns = ['uniform', 'clustered']
sizes = [50, 100, 150]

pattern_results = []

for pattern in patterns:
    for size in sizes:
        print(f"Testing {pattern} {size}x{size}...")
        
        if pattern == 'uniform':
            grid = GridGenerator.generate_random(size, density=0.25, seed=42)
        else:
            grid = GridGenerator.generate_clustered(size, density=0.25, seed=42)
        
        try:
            results = run_benchmark(grid, num_pairs=20, seed=42)
            stats = compute_statistics(results)
            
            for method, s in stats.items():
                pattern_results.append({
                    'pattern': pattern,
                    'grid_size': size,
                    'method': method,
                    'time_mean': s['time_mean'],
                    'nodes_mean': s['nodes_mean']
                })
        except:
            pass

df_pattern = pd.DataFrame(pattern_results)

In [ ]:
# Create heatmap
if len(df_pattern) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    for idx, metric in enumerate(['time_mean', 'nodes_mean']):
        ax = axes[idx]
        
        # Pivot for AILS-Adaptive
        df_adaptive = df_pattern[df_pattern['method'] == 'ails_adaptive']
        
        if len(df_adaptive) > 0:
            pivot = df_adaptive.pivot(index='pattern', columns='grid_size', values=metric)
            
            sns.heatmap(pivot, annot=True, fmt='.1f', cmap='YlOrRd', ax=ax)
            
            title = 'Time (ms)' if metric == 'time_mean' else 'Nodes Visited'
            ax.set_title(f'AILS-Adaptive: {title}')
            ax.set_xlabel('Grid Size')
            ax.set_ylabel('Pattern')
    
    plt.tight_layout()
    fig.savefig(f'{FIGURES_DIR}/pattern_heatmap.png', bbox_inches='tight')
    plt.show()

## 6. Algorithm Flowchart (Conceptual)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Define boxes
boxes = [
    (0.5, 0.9, 'Input: Start, Goal, Grid'),
    (0.5, 0.75, 'Bresenham Line\nGeneration'),
    (0.5, 0.6, 'Local Density\nEstimation'),
    (0.5, 0.45, 'Adaptive Radius\nComputation'),
    (0.5, 0.3, 'Corridor\nConstruction'),
    (0.5, 0.15, 'Constrained A*\nSearch'),
    (0.5, 0.02, 'Output: Path')
]

# Draw boxes
for x, y, text in boxes:
    bbox = dict(boxstyle='round,pad=0.3', facecolor='lightblue', edgecolor='navy')
    ax.text(x, y, text, ha='center', va='center', fontsize=11,
            bbox=bbox, transform=ax.transAxes)

# Draw arrows
for i in range(len(boxes) - 1):
    ax.annotate('', xy=(0.5, boxes[i+1][1] + 0.05), xytext=(0.5, boxes[i][1] - 0.05),
               xycoords='axes fraction', textcoords='axes fraction',
               arrowprops=dict(arrowstyle='->', color='navy', lw=2))

# Expansion loop
ax.annotate('', xy=(0.75, 0.35), xytext=(0.75, 0.18),
           xycoords='axes fraction', textcoords='axes fraction',
           arrowprops=dict(arrowstyle='->', color='red', lw=1.5,
                          connectionstyle='arc3,rad=-0.3'))
ax.text(0.82, 0.26, 'Expand if\npath not found', fontsize=9, color='red',
       transform=ax.transAxes)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('AILS Algorithm Flowchart', fontsize=14, fontweight='bold')

plt.tight_layout()
fig.savefig(f'{FIGURES_DIR}/algorithm_flowchart.png', bbox_inches='tight')
plt.show()

## 7. Summary Statistics Table

In [ ]:
if len(df_scaling) > 0:
    # Create summary table figure
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.axis('off')
    
    # Prepare data
    summary_data = df_scaling.groupby('method').agg({
        'time_mean': 'mean',
        'nodes_mean': 'mean'
    }).round(2)
    
    labels = {'standard_astar': 'Standard A*', 'ails_base': 'AILS-Base', 'ails_adaptive': 'AILS-Adaptive'}
    summary_data.index = [labels.get(m, m) for m in summary_data.index]
    summary_data.columns = ['Avg Time (ms)', 'Avg Nodes']
    
    # Create table
    table = ax.table(cellText=summary_data.values,
                    rowLabels=summary_data.index,
                    colLabels=summary_data.columns,
                    cellLoc='center',
                    loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.2, 1.5)
    
    ax.set_title('AILS Performance Summary', fontsize=14, fontweight='bold', pad=20)
    
    fig.savefig(f'{FIGURES_DIR}/summary_table.png', bbox_inches='tight')
    plt.show()

In [ ]:
# List all generated figures
print("\n" + "="*70)
print("GENERATED FIGURES")
print("="*70)

for f in sorted(os.listdir(FIGURES_DIR)):
    if f.endswith('.png'):
        path = os.path.join(FIGURES_DIR, f)
        size = os.path.getsize(path) / 1024
        print(f"  {f} ({size:.1f} KB)")

print("\nVisualization complete!")